In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


# Check if CUDA is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if not torch.cuda.is_available():
    quit()

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(64 * 6 * 6, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = Net()
net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Training
for epoch in range(10):
    print(f"Epoch {epoch + 1}")
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# # FGSM attack
# def fgsm_attack(image, epsilon, data_grad):
#     sign_data_grad = data_grad.sign()
#     perturbed_image = image + epsilon * sign_data_grad
#     perturbed_image = torch.clamp(perturbed_image, 0, 1)
#     return perturbed_image

# def test_attack(model, test_loader, epsilon):
#     for data, target in test_loader:
#         data.requires_grad = True
#         output = model(data)
#         init_pred = output.max(1, keepdim=True)[1]

#         loss = criterion(output, target)
#         model.zero_grad()
#         loss.backward()
#         data_grad = data.grad.data
#         perturbed_data = fgsm_attack(data, epsilon, data_grad)

#         output = model(perturbed_data)
#         final_pred = output.max(1, keepdim=True)[1]
#         if final_pred.item() != target.item():
#             return True  # Attack successful
#     return False

# epsilon = 0.01
# success = test_attack(net, testloader, epsilon)
# print(f"Attack success: {success}")







Using device: cpu
Epoch 1
Epoch 2


In [ ]:
def rgb_noise_attack(image, epsilon):
    """
    Add random noise in the range [-epsilon, epsilon) to each pixel's RGB value.
    """
    noise = (2 * torch.rand(image.shape) - 1) * epsilon
    perturbed_image = image + noise
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

def test_rgb_attack(model, test_loader, epsilon):
    for data, target in test_loader:
        perturbed_data = rgb_noise_attack(data, epsilon)

        output = model(perturbed_data)
        final_pred = output.max(1, keepdim=True)[1]
        for i in range(final_pred.size(0)):  # Iterate over each item in the batch
            if final_pred[i].item() != target[i].item():
                print(f"Predicted {final_pred[i].item()}, actual {target[i].item()}")
                return True  # Attack successful
    return False

epsilon = 0.1  # You might want to adjust this value to see different effects
success = test_rgb_attack(net, testloader, epsilon)
print(f"Attack success: {success}")